In [1]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords

Очистка текста включает в себя:

Удаление лишних символов (например, пунктуации, специальные символы).
Приведение текста к нижнему регистру.
Удаление стоп-слов (неинформативных слов).

In [2]:
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))

def clean_text(text):
    # Удаляем специальные символы и приводим к нижнему регистру
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    
    # Удаляем стоп-слова
    text = ' '.join(word for word in text.split() if word not in stop_words)
    
    return text

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\an23m\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


После очистки текста можно разбить его на отдельные слова (токены):

In [3]:
def tokenize(text):
    return text.split()

Лемматизация или стемминг
Вы можете использовать nltk или pymorphy2 для лемматизации:

In [6]:
import pymorphy2

morph = pymorphy2.MorphAnalyzer()

def lemmatize(tokens):
    return [morph.parse(token)[0].normal_form for token in tokens]

In [7]:
def preprocess_text(text):
    cleaned = clean_text(text)
    tokens = tokenize(cleaned)
    lemmatized = lemmatize(tokens)
    return lemmatized

Применение предобработки к данным

In [13]:
# Пример извлечения данных из базы и применения предобработки
import psycopg2

# Установите соединение с вашей базой данных
conn = psycopg2.connect("dbname=azs user=postgres password=f8ysz789")
cur = conn.cursor()

cur.execute("SELECT comment_text FROM buffer.azs_review limit 1;")
rows = cur.fetchall()

preprocessed_reviews = [preprocess_text(row[0]) for row in rows]

# Не забудьте закрыть соединение
cur.close()
conn.close()

In [14]:
preprocessed_reviews

[['заехать',
  'дорога',
  'покушать',
  '9',
  'вечер',
  'очень',
  'вежливый',
  'обслуживание',
  'улыбчивый',
  'доброжелательный',
  'персонал',
  'вкусный',
  'еда',
  'взять',
  'удон',
  'карбонар',
  'чизбургер',
  'всё',
  'понравиться',
  'туалет',
  'чистый',
  'бумага',
  'мыло',
  'наличие',
  'приятный',
  'остановка',
  'путь',
  'домой']]